In [1]:
from extract_data import run_etl
import pandas as pd

In [2]:
def update_names():
    # get all dataframes as a dictionary of (df, table_name)
    dataframes = run_etl()

    # Create a dictionary to hold the dataframes
    df_dict = {}
    for df, table_name in dataframes:
        df_dict[f"df_{table_name}"] = df
    return df_dict

In [ ]:
# Change the working directory (If this is changed, the following cells will crash because it won't find the data folder)
import os

# Define the relative path
target_path = './../data'
move_path = "./../"

# Check if the directory exists before changing
if os.path.exists(target_path):
    os.chdir(move_path)
    print(f"Changed directory to: {os.getcwd()}")


In [4]:
df_dict = update_names()

display_table = "df_section" # Change to display table

In [ ]:
# Remove rows with null values across all dataframes
df_dict["df_class"].dropna(inplace=True)
df_dict["df_section"].dropna(inplace=True)
df_dict["df_meeting"].dropna(inplace=True)
df_dict["df_room"].dropna(inplace=True)
df_dict["df_requisite"].dropna(inplace=True)

# Init the dataframes
df_class = df_dict["df_class"]
df_section = df_dict["df_section"]
df_meeting = df_dict["df_meeting"]
df_room = df_dict["df_room"]
df_requisite = df_dict["df_requisite"]

display(df_dict[display_table])

### 1. Ensure that classes have ID starting from 2

In [ ]:
df_class["cid"] = pd.to_numeric(df_class["cid"], errors="coerce")
df_class.dropna(subset=["cid"], inplace=True)
df_class = df_class[df_class["cid"] >= 2]
df_section = df_section[df_section["cid"] >= 2]

print(f"Length of df_section: {len(df_section)}")

#### 7. Sections cannot be in overcapacity, classrooms have limits.


In [ ]:
df_section_room = df_section.merge(df_room, left_on="roomid", right_on="rid")
df_section_room = df_section_room[
    df_section_room["capacity_x"] <= df_section_room["capacity_y"]
]
df_section = df_section[df_section["sid"].isin(df_section_room["sid"])]

print(f"Length of df_section: {len(df_section)}")


#### 8. Courses must be taught in the correct year and correct semester.

In [ ]:

df_section_class = df_section.merge(df_class, on="cid")
years_x = pd.to_numeric(df_section_class["years_x"], errors="coerce")

# Boolean Conditions
First_semester = (
    (df_section_class["term"] == "First Semester")
    | (df_section_class["term"] == "First Semester, Second Semester")
) & (df_section_class["semester"] == "Fall")

Second_semester = (
    (df_section_class["term"] == "Second Semester")
    | (df_section_class["term"] == "First Semester, Second Semester")
) & (df_section_class["semester"] == "Spring")

According_Demand = (df_section_class["term"] == "According to Demand") & (
    (df_section_class["semester"] == "Fall")
    | (df_section_class["semester"] == "Spring")
    | (df_section_class["semester"] == "V1")
    | (df_section_class["semester"] == "V2")
)

Even_year = (df_section_class["years_y"] == "Even Years") & ((years_x % 2) == 0)
Odd_year = (df_section_class["years_y"] == "Odd Years") & ((years_x % 2) != 0)
Every_Year = df_section_class["years_y"] == "Every Year"
According_Demand_Year = df_section_class["years_y"] == "According to Demand"

# Combine the boolean conditions into a single series
combined_conditions = (First_semester | Second_semester | According_Demand) & (
    Even_year | Odd_year | Every_Year | According_Demand_Year
)

# Ensure the combined_conditions series has the same index as df_section_class
combined_conditions = combined_conditions.reindex(df_section_class.index)

# Set pandas option to display all rows
pd.set_option('display.max_rows', None)

columns_to_display = ['semester', 'years_x', 'term', 'years_y']
# Display rows that do NOT satisfy the combined_conditions
display(df_section_class.loc[~combined_conditions, columns_to_display])

# Reset pandas option back to the default after displaying all rows
pd.reset_option('display.max_rows')

# Filter the sections based on the combined boolean conditions
df_section_class = df_section_class[combined_conditions]


# Update the section dataframe
df_section = df_section[df_section["sid"].isin(df_section_class["sid"])]
# display(df_section_class)

print(f"Length of df_section: {len(df_section)}")

#### 2. Two sections cannot be taught at the same hour in the same classroom. 
#### 3. A class cannot have the same section, they must be taught at different hours.

In [ ]:
from IPython.display import display_html

# Delete sections with duplicate 'sid'
df_section = df_section.drop_duplicates(subset=["sid"], keep=False)

# Merge 'section' with 'meeting' to check for overlapping sections in the same room, time, and semester
df_section_meeting = df_section.merge(df_meeting, on="mid")

# Convert 'starttime' and 'endtime' to time objects
df_section_meeting["starttime"] = pd.to_datetime(
    df_section_meeting["starttime"], format="%H:%M:%S"
).dt.time
df_section_meeting["endtime"] = pd.to_datetime(
    df_section_meeting["endtime"], format="%H:%M:%S"
).dt.time

# Sort the dataframe by room, semester, starttime, and sid
df_section_meeting = df_section_meeting.sort_values(
    ["roomid", "semester", "starttime", "sid"]
)

# Detect overlapping sections (same room, same semester, same time)
overlaps = []
repited = []
for (roomid, semester, starttime, cdays), group in df_section_meeting.groupby(
    ["roomid", "semester", "starttime", "cdays"]
):  
    
    for i in range(1, len(group)):
        previous = group.iloc[i - 1]
        current = group.iloc[i]

        # Check if sections overlap in the same room, same time, and same semester
        if (
            current["starttime"] == previous["starttime"]
            and current["roomid"] == previous["roomid"]
            and current["semester"] == previous["semester"]
            and current["years"] == previous["years"]
            and current["cdays"] == previous["cdays"]
        ):
            # Add the section with the higher sid to the list of overlaps to delete
            if current["sid"] > previous["sid"]:
                overlaps.append(current["sid"])
            else:
                overlaps.append(previous["sid"])

            repited.append(current["sid"])
            repited.append(previous["sid"])


df_overlaps = df_section_meeting[df_section_meeting["sid"].isin(overlaps)]
df_repited = df_section_meeting[df_section_meeting["sid"].isin(repited)]

df_repited_html = df_repited.to_html()
df_overlaps_html = df_overlaps.to_html()

# Display the two DataFrames side by side using HTML tables
display_html(f"<div style='display: flex; justify-content: space-around;'>"
             f"<div>{df_repited_html}</div><div>{df_overlaps_html}</div></div>", raw=True)
# Remove overlapping sections with the higher sid
df_section = df_section[~df_section["sid"].isin(overlaps)]

print(f"Length of df_section: {len(df_section)}")



#### 4. Adjust 'MJ' meetings and remove overlaps


In [ ]:
from IPython.display import display_html

# Function to convert 'HH:MM:SS' format to minutes
def convert_to_minutes(time_str):
    hours, minutes, _ = map(int, time_str.split(':'))
    return hours * 60 + minutes

# Function to convert minutes back to 'HH:MM' format
def convert_to_hhmm(total_minutes):
    hours = total_minutes // 60
    minutes = total_minutes % 60
    return f'{hours}:{minutes:02d}' 

# Make a deep copy of df_meeting to preserve original data
df_meeting_prev = df_meeting.copy(deep=True)

# Convert 'starttime' and 'endtime' columns to minutes for easier manipulation
df_meeting[['starttime', 'endtime']] = df_meeting[['starttime', 'endtime']].applymap(convert_to_minutes)

# Filter out meetings on 'MJ' days between 10:15 and 12:30
df_meeting = df_meeting[
    ~(
        (df_meeting['cdays'] == "MJ") & 
        (df_meeting['starttime'] > convert_to_minutes("10:15:00")) & 
        (df_meeting['endtime'] < convert_to_minutes("12:30:00"))
    )
]

# Function to find the index of the earliest and latest class based on condition
def find_class_index(condition):
    try:
        return df_meeting[condition].index[0]
    except IndexError:
        return -1

# Find earliest and latest class times between 10:15 and 12:30 on 'MJ' days
index_earliest_class_after_1030 = find_class_index(
    (df_meeting['cdays'] == "MJ") & 
    (df_meeting['starttime'] > convert_to_minutes("10:15:00")) & 
    (df_meeting['starttime'] < convert_to_minutes("12:30:00"))
)

index_latest_class_before_1230 = find_class_index(
    (df_meeting['cdays'] == "MJ") & 
    (df_meeting['endtime'] < convert_to_minutes("12:30:00")) & 
    (df_meeting['endtime'] > convert_to_minutes("10:15:00"))
)

# Adjust class timings if valid indices are found
if index_earliest_class_after_1030 != -1:
    delta_time = convert_to_minutes("12:30:00") - df_meeting.loc[index_earliest_class_after_1030, 'starttime']
    df_meeting.loc[(df_meeting['cdays'] == "MJ") & (df_meeting.index >= index_earliest_class_after_1030), ['starttime', 'endtime']] += delta_time

if index_latest_class_before_1230 != -1:
    delta_time = df_meeting.loc[index_earliest_class_after_1030, 'endtime'] - convert_to_minutes("10:15:00")
    df_meeting.loc[(df_meeting['cdays'] == "MJ") & (df_meeting.index <= index_latest_class_before_1230), ['starttime', 'endtime']] -= delta_time

# Remove all meetings that start after 19:45
df_meeting = df_meeting[df_meeting["starttime"] <= convert_to_minutes("19:45:00")]

# Convert 'starttime' and 'endtime' columns back to 'HH:MM' format
df_meeting[['starttime', 'endtime']] = df_meeting[['starttime', 'endtime']].applymap(convert_to_hhmm)

# Convert 'starttime' and 'endtime' back to datetime.time format for display purposes
df_meeting["starttime"] = pd.to_datetime(df_meeting["starttime"], format="%H:%M").dt.time
df_meeting["endtime"] = pd.to_datetime(df_meeting["endtime"], format="%H:%M").dt.time

# Create HTML representations of the two DataFrames for side-by-side comparison
df_meeting_html = df_meeting.to_html()
df_meeting_prev_html = df_meeting_prev.to_html()

# Display the two DataFrames side by side using HTML tables
display_html(f"<div style='display: flex; justify-content: space-around;'>"
             f"<div>{df_meeting_prev_html}</div><div>{df_meeting_html}</div></div>", raw=True)

print(f"Length of df_section: {len(df_section)}")


####  5. All ‘LWV’ sections have the correct hours
#### 6. ‘LWV’ meetings have a duration of 50 minutes; ‘MJ’ meetings have a duration of 75 minutes.


In [ ]:
df_meeting.loc[df_meeting["cdays"] == "LWV", "duration"] = 50
df_meeting.loc[df_meeting["cdays"] == "MJ", "duration"] = 75

df_meeting = df_meeting[
    (
        (df_meeting["cdays"] == "LWV")
        & (
            pd.to_datetime(df_meeting["endtime"], format="%H:%M:%S")
            - pd.to_datetime(df_meeting["starttime"], format="%H:%M:%S")
            == pd.Timedelta(minutes=50)
        )
    )
    | (
        (df_meeting["cdays"] == "MJ")
        & (
            pd.to_datetime(df_meeting["endtime"], format="%H:%M:%S")
            - pd.to_datetime(df_meeting["starttime"], format="%H:%M:%S")
            == pd.Timedelta(minutes=75)
        )
    )
]

df_meeting.drop(columns=["duration"], inplace=True)

print(f"Length of df_section: {len(df_section)}")


#### 9. Sections must be taught in a valid classroom and meeting, and the class must exist.


In [ ]:
df_section = df_section[df_section["roomid"].isin(df_room["rid"])]
df_section = df_section[df_section["mid"].isin(df_meeting["mid"])]
df_section = df_section[df_section["cid"].isin(df_class["cid"])]
print(f"Length of df_section: {len(df_section)}")


#### 10. Delete all section with Dummy class as Foreign Key


In [ ]:
dummy_class_ids = df_class[
    df_class["cname"] == "Authorization from the Director of the Department"
]["cid"].tolist()

df_section = df_section[~df_section["cid"].isin(dummy_class_ids)]
print(f"Length of df_section: {len(df_section)}")

In [ ]:
# Dsplay Cleaned Section DataFrame
print(f"Length of df_section: {len(df_section)}")
print(f"Length of df_class: {len(df_class)}")
print(f"Length of df_meeting: {len(df_meeting)}")
print(f"Length of df_room: {len(df_room)}")
print(f"Length of df_requisite: {len(df_requisite)}")
print(f"Total length of all dataframes: {len(df_section) + len(df_class) + len(df_meeting) + len(df_room) + len(df_requisite)}")

pd.set_option('display.max_rows', None)
display(df_section)
pd.reset_option('display.max_rows')
# display(df_class)
# display(df_meeting)
# display(df_room)
# display(df_requisite)
